# Import libraries

In [12]:
import pandas as pd
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller

from linearmodels import PanelOLS, RandomEffects

from stargazer.stargazer import Stargazer
from statsmodels.formula.api import ols

#!pip install patsy
#!pip install transliterate
from transliterate import translit

from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel

import pandas_datareader as dr

import lightgbm as lgbm
from pandas.plotting import scatter_matrix

import random
random.seed(13)

from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform
from scipy.stats import loguniform

from stargazer.stargazer import Stargazer
from IPython.core.display import HTML

import warnings
warnings.filterwarnings('ignore')

# Download data

In [13]:
data = pd.read_csv('final_data_without_prefs_and_with_HAMADA_factors.csv')
data_ret = pd.read_csv('data_ret.csv')
data_ret.drop(columns=['Unnamed: 0'], inplace=True)
#data_ret.set_index('Date', inplace=True)
data.set_index('Date', inplace = True)
macro_data = pd.read_csv('macro_data.csv')
macro_data.set_index('Date', inplace = True)

# Get the results

## Sber is an example, for any other company you have to change SHARE_INDEX

### Companies with their numbers

### SBER - 0, GAZP - 1, LKOH - 2, GMKN - 3, NVTK - 4, SNGS - 5, PLZL - 6, TATN - 7, ROSN - 8, MGNT - 9, MTSS - 10, MOEX - 11, IRAO - 12, NLMK - 13, ALRS - 14, CHMF - 15, VTBR - 16, RTKM - 17, PHOR - 18, AFKS - 19, MAGN - 20, PIKK - 21, HYDR - 22, FEES - 23, AFLT - 24, LSRG - 25, RSTI - 26, UPRO - 27, LNTA - 28, MVID - 29, TRMK - 30, MTLR - 31, NMTP - 32, AKRN - 33, MSNG - 34, URKA - 35, BANE - 36, VSMO - 37, NKNC - 38, GCHE - 39, SVAV - 40, MSTT - 41, BSPB - 42, MSRS - 43, KMAZ - 44, RASP - 45, OGKB - 46, VZRZ - 47

In [14]:
SHARE_INDEX = 0

In [15]:
lag_autolag_return_30 = 30
lag_autolag_return_60 = 60
lag_autolag_return_90 = 90
lag_30 = 30
lag_60 = 60
lag_90 = 90

Final_results_for_companies = []
for name in data_ret.columns[2:]:

    df1 = data[[name, 'Hamada_beta_MRP_for_{}'.format(name[7:11]), 'risk_free_rate_based_on_g_curve',
                 'Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]),
                 'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]),
                 'MOM']].dropna()
    df_autolag = pd.DataFrame(data[name] - data['risk_free_rate_based_on_g_curve']).rename(columns = {0:'{}_premium_lag_{}_days'.format(name[7:], lag_autolag_return_30)})
    df_autolag = pd.DataFrame(df_autolag['{}_premium_lag_{}_days'.format(name[7:], lag_autolag_return_30)].shift(lag_autolag_return_30).dropna())

    df_autolag_2 = pd.DataFrame(data[name] - data['risk_free_rate_based_on_g_curve']).rename(columns = {0:'{}_premium_lag_{}_days'.format(name[7:], lag_autolag_return_60)})
    df_autolag_2 = pd.DataFrame(df_autolag_2['{}_premium_lag_{}_days'.format(name[7:], lag_autolag_return_60)].shift(lag_autolag_return_60).dropna())

    df_autolag_3 = pd.DataFrame(data[name] - data['risk_free_rate_based_on_g_curve']).rename(columns = {0:'{}_premium_lag_{}_days'.format(name[7:], lag_autolag_return_90)})
    df_autolag_3 = pd.DataFrame(df_autolag_3['{}_premium_lag_{}_days'.format(name[7:], lag_autolag_return_90)].shift(lag_autolag_return_90).dropna())

    df2 = data[['Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]),
            'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]), 'MOM']].shift(lag_30).dropna()



    df2.rename(columns = {'Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]):'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_30),
                          'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]):'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_30), 
                          'MOM':'MOM_lag_{}_days'.format(lag_30)}, inplace = True)

    result = pd.concat([df1, df2], axis=1, join="inner")

    df3 = data[['Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]),
            'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]), 'MOM']].shift(lag_60).dropna()

    df3.rename(columns = {'Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]):'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_60),
                          'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]):'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_60), 
                          'MOM':'MOM_lag_{}_days'.format(lag_60)}, inplace = True)

    result = pd.concat([result, df3], axis=1, join="inner")

    df4 = data[['Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]),
            'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]), 'MOM']].shift(lag_90).dropna()

    df4.rename(columns = {'Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]):'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_90),
                          'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]):'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_90), 
                          'MOM':'MOM_lag_{}_days'.format(lag_90)}, inplace = True)

    result = pd.concat([result, df4], axis=1, join="inner")

    result = pd.merge(result, macro_data, left_index=True, right_index=True)
    result = pd.merge(result, df_autolag, left_index=True, right_index=True)
    result = pd.merge(result, df_autolag_2, left_index=True, right_index=True)
    result = pd.merge(result, df_autolag_3, left_index=True, right_index=True)


    Base = result.dropna()

    Y = pd.DataFrame(Base[name] - Base['risk_free_rate_based_on_g_curve'])
    Y.rename(columns = {0:'{} - risk free rate'.format(name)}, inplace = True)
    X = Base[['Hamada_beta_MRP_for_{}'.format(name[7:11]), 
              '{}_premium_lag_{}_days'.format(name[7:], lag_autolag_return_30),
              '{}_premium_lag_{}_days'.format(name[7:], lag_autolag_return_60),
              '{}_premium_lag_{}_days'.format(name[7:], lag_autolag_return_90),
              'Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]),
              'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]), 
              'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_30),
              'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_30),
              'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_60),
              'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_60),
              'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_90),
              'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_90),
              'MOM','MOM_lag_{}_days'.format(lag_30),'MOM_lag_{}_days'.format(lag_60),
              'MOM_lag_{}_days'.format(lag_90),'dollar_30d_return','wti_30d_return']]
    Y.rename(columns = {'{} - risk free rate'.format(name):'{}_minus_rf'.format(name[7:11])}, inplace = True)

    dataC = pd.concat([X, Y], axis=1, join="inner")
    
    # technical column renaming:
    column_list_new = [ translit(''.join([word.capitalize().replace("." ,"") for word in col.split(' ') ]), "ru", reversed=True).replace("'", "").replace("-", "") for col in dataC.columns]
    regressionRenameDict = dict(zip(dataC.columns, column_list_new ))

    data_reg = dataC.rename(columns= regressionRenameDict, inplace=False)
    
    #### CAPM
    
    OLS1 = sm.OLS.from_formula(formula= """{} ~ {} - 1""".format(data_reg.columns[-1], data_reg.columns[0]), data= data_reg) 
    OLS1_fit = OLS1.fit(cov_type = "HC3")
    
    #### Fama-French
    
    OLS2 = sm.OLS.from_formula(formula= """{} ~ {} + {} + {} - 1""".format(data_reg.columns[-1],
                                        data_reg.columns[0], data_reg.columns[4], data_reg.columns[5]), data= data_reg)
    OLS2_fit = OLS2.fit(cov_type = "HC3")
    
    #### Carhart
    
    OLS3 = sm.OLS.from_formula(formula= """{} ~ {} + {} + {} + {} - 1""".format(data_reg.columns[-1],
                                        data_reg.columns[0], data_reg.columns[4], data_reg.columns[5],
                                        data_reg.columns[-7]), data= data_reg)
    OLS3_fit = OLS3.fit(cov_type = "HC3")
    
    #### Carhart with lags
    
    OLS4 = sm.OLS.from_formula(formula= """{} ~ {} + {} + {} + {} + {} + {} +
                                        {} + {} + {} + {} + {} + {} + {} - 1""".format(data_reg.columns[-1],
                                        data_reg.columns[0], data_reg.columns[4], data_reg.columns[5],
                                        data_reg.columns[6], data_reg.columns[7], data_reg.columns[8], 
                                        data_reg.columns[9], data_reg.columns[10], data_reg.columns[11],
                                        data_reg.columns[-7], data_reg.columns[13], data_reg.columns[14],
                                        data_reg.columns[15]), data= data_reg)
    OLS4_fit = OLS4.fit(cov_type = "HC3")
    
    #### Carhart with lags and autolags
    
    OLS5 = sm.OLS.from_formula(formula= """{} ~ {} + {} + {} + {} + {} + {} +
                                        {} + {} + {} + {} + {} + {} + {} + {} + {} + {} - 1""".format(data_reg.columns[-1],
                                        data_reg.columns[0], data_reg.columns[1], data_reg.columns[2], 
                                        data_reg.columns[3], data_reg.columns[4], data_reg.columns[5],
                                        data_reg.columns[6], data_reg.columns[7], data_reg.columns[8], 
                                        data_reg.columns[9], data_reg.columns[10], data_reg.columns[11],
                                        data_reg.columns[-7], data_reg.columns[13], data_reg.columns[14],
                                        data_reg.columns[15]), data= data_reg)
    OLS5_fit = OLS5.fit(cov_type = "HC3")
    
    #### Carhart with lags and autolags + specific for Russia
    
    OLS6 = sm.OLS.from_formula(formula= """{} ~ {} + {} + {} + {} + {} + {} +
                                        {} + {} + {} + {} + {} + {} + {} + {} + {} + {} + {} + {} - 1""".format(data_reg.columns[-1],
                                        data_reg.columns[0], data_reg.columns[1], data_reg.columns[2], 
                                        data_reg.columns[3], data_reg.columns[4], data_reg.columns[5],
                                        data_reg.columns[6], data_reg.columns[7], data_reg.columns[8], 
                                        data_reg.columns[9], data_reg.columns[10], data_reg.columns[11],
                                        data_reg.columns[-7], data_reg.columns[13], data_reg.columns[14],
                                        data_reg.columns[15], data_reg.columns[-3], data_reg.columns[-2]), data= data_reg)
    OLS6_fit = OLS6.fit(cov_type = "HC3")
    
    
    
    #### into Stargazer
    
    res = Stargazer([OLS1_fit, OLS2_fit, OLS3_fit,OLS4_fit, OLS5_fit, OLS6_fit])
    res.covariate_order(['{}'.format(data_reg.columns[0]),
                         '{}'.format(data_reg.columns[1]), 
                         '{}'.format(data_reg.columns[2]),
                         '{}'.format(data_reg.columns[3]),
                         '{}'.format(data_reg.columns[4]), 
                         '{}'.format(data_reg.columns[5]),
                         '{}'.format(data_reg.columns[6]),
                         '{}'.format(data_reg.columns[7]),
                         '{}'.format(data_reg.columns[8]), 
                         '{}'.format(data_reg.columns[9]),
                         '{}'.format(data_reg.columns[10]),
                         '{}'.format(data_reg.columns[11]),
                         '{}'.format(data_reg.columns[12]),
                         '{}'.format(data_reg.columns[13]),
                         '{}'.format(data_reg.columns[14]),
                         '{}'.format(data_reg.columns[15]),
                         '{}'.format(data_reg.columns[-3]),
                         '{}'.format(data_reg.columns[-2])])
    
    
    res.rename_covariates({'{}'.format(data_reg.columns[0]):'Премия за рыночный риск',
                           '{}'.format(data_reg.columns[1]):'доходность {} лаг месяц'.format(data_reg.columns[1][:4]),
                           '{}'.format(data_reg.columns[2]):'доходность {} лаг 2 месяца'.format(data_reg.columns[2][:4]),
                           '{}'.format(data_reg.columns[3]):'доходность {} лаг 3 месяца'.format(data_reg.columns[3][:4]),
                           '{}'.format(data_reg.columns[4]):'SMB',
                           '{}'.format(data_reg.columns[5]):'HML',
                           '{}'.format(data_reg.columns[6]):'SMB лаг месяц',
                           '{}'.format(data_reg.columns[7]):'HML лаг месяц',
                           '{}'.format(data_reg.columns[8]):'SMB лаг 2 месяца', 
                           '{}'.format(data_reg.columns[9]):'HML лаг 2 месяца',
                           '{}'.format(data_reg.columns[10]):'SMB лаг 3 месяца',
                           '{}'.format(data_reg.columns[11]):'HML лаг 3 месяца',
                           '{}'.format(data_reg.columns[-7]):'MOM',
                           '{}'.format(data_reg.columns[13]):'MOM лаг месяц',
                           '{}'.format(data_reg.columns[14]):'MOM лаг 2 месяца',
                           '{}'.format(data_reg.columns[15]):'MOM лаг 3 месяца',
                           '{}'.format(data_reg.columns[-3]):'Изменение курса доллара за месяц',
                           '{}'.format(data_reg.columns[-2]):'Изменение цены на нефть за месяц'})
    
    res.title('{}'.format(data_reg.columns[2][:4].upper()))
    res.significant_digits(3)
    res.custom_columns(['CAPM', 'Фама-Френч', 'Кархарт', 'Кархарт + лаги', 
                             'Кархарт + лаги и автолаги', 'Кархарт + лаги и автолаги + специфические'], [1, 1, 1, 1, 1, 1])
    res.add_custom_notes(['Во всех моделях учтено наличие долга у компаний по формуле Р.Хамады для рыночной премии за риск, SMB, HML'])
    
    Final_results_for_companies.append(res)
    
#View the results
Final_results_for_companies[SHARE_INDEX]    
